# archive to parquet

Convert a remote archive or csv file (or local file://), to parquet format

In [1]:
import mlrun
import os
mlrun.mlconf.dbpath = 'http://mlrun-api:8080'

In [2]:
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

## parameters


In [15]:
BASE_IMAGE         = 'yjbds/mlrun-files:latest'

CODE_BASE          = '/User/repos/functions/'
PROJECT            = 'fileutils/arc_to_parquet'

TARGET_PATH        = '/User/mlrun/airlines/dataset'

ARCHIVE_BIG        = "https://s3.amazonaws.com/h2o-airlines-unpacked/allyears_10.csv"
ARCHIVE            = "https://s3.amazonaws.com/h2o-airlines-unpacked/allyears.csv"
ARCHIVE_SMALL      = "https://s3.amazonaws.com/h2o-airlines-unpacked/allyears2k.csv"

**For testing and development use ARCHIVE_SMALL:**

In [4]:
USE_ARCHIVE = ARCHIVE

In [19]:
FILE_NAME          = 'airlines.pqt'
KEY                = 'airlines'

# no need for this as the files contain a header:
HEADER = ['Year','Month','DayofMonth','DayOfWeek','DepTime','CRSDepTime','ArrTime','CRSArrTime',
          'UniqueCarrier','FlightNum','TailNum','ActualElapsedTime','CRSElapsedTime','AirTime',
          'ArrDelay','DepDelay','Origin','Dest','Distance','TaxiIn','TaxiOut','Cancelled',
          'CancellationCode','Diverted','CarrierDelay','WeatherDelay','NASDelay','SecurityDelay',
          'LateAircraftDelay']
INC_COLS = ['Year','Month','DayofMonth','DayOfWeek','DepTime','CRSDepTime','ArrTime','CRSArrTime',
          'UniqueCarrier','FlightNum', 'CRSElapsedTime','AirTime',
          'Origin','Dest','Distance','TaxiOut','Cancelled',
          'CarrierDelay','WeatherDelay','NASDelay','SecurityDelay',
          'LateAircraftDelay']

ENCODING = 'latin-1'

DTYPES_COLS = {
     'CRSElapsedTime': 'float64', 
     'TailNum': 'str', 
     'Distance': 'float64', 
     'TaxiOut': 'float64',
     'ArrTime': 'float64',
     'DepTime':'float64', 
     'CarrierDelay': 'float64', 
     'WeatherDelay': 'float64', 
     'NASDelay':'float64', 
     'SecurityDelay':'float64', 
     'LateAircraftDelay':'float64'}

USE_PARTITIONS = True
PARTITION_COLS = ['Year', 'Month']

In [6]:
os.makedirs(TARGET_PATH, exist_ok=True)

## load and configure function

**If run the first time, create the function:**

In [7]:
# load function from a local Python file
arctoparq = mlrun.code_to_function(
    filename=os.path.join(CODE_BASE, PROJECT, 'arc_to_parquet.py'), 
    kind='job')
arctoparq.build_config(base_image=BASE_IMAGE, commands=[])
yaml_name = os.path.join(CODE_BASE, PROJECT, 'arc_to_parquet.yaml')
arctoparq.export(yaml_name)

[mlrun] 2020-01-27 19:05:25,830 function spec saved to path: /User/repos/functions/fileutils/arc_to_parquet/arc_to_parquet.yaml


**otherwise load it:**

In [8]:
arctoparq = mlrun.import_function(
    os.path.join(CODE_BASE, PROJECT, 'arc_to_parquet.yaml')
).apply(mlrun.mount_v3io())

## deploy / build

The following triggers a build when run for the first time using specs found in the yaml file above.

In [9]:
arctoparq.deploy(skip_deployed=True, with_mlrun=False)

'ready'

In [ ]:
%%time
# create and run the task
arc_to_parq_task = mlrun.NewTask(
    'arc2parq', 
    handler='arc_to_parquet',  
    params={
        'target_path': TARGET_PATH,
        'name'       : FILE_NAME, 
        'key'        : KEY,
        'archive_url': USE_ARCHIVE,
        'dataset'    : USE_PARTITIONS,
        'part_cols'  : PARTITION_COLS,
        'encoding'   : ENCODING,
        'inc_cols'   : INC_COLS,
        'dtype'      : DTYPES_COLS})
# run
run = arctoparq.run(arc_to_parq_task)

[mlrun] 2020-01-27 19:21:20,254 starting run arc2parq uid=d3a5446edb94436d91efe4b2d7c64c2b  -> http://mlrun-api:8080
[mlrun] 2020-01-27 19:21:20,379 Job is running in the background, pod: arc2parq-5qs84


___

## tests

### a partitioned parquet table

In [16]:
dataset = pq.ParquetDataset(TARGET_PATH)

ValueError: Schema in /User/mlrun/airlines/dataset/1278d2c85afc40cabc8e5add8d12892e.parquet was different. 
Year: int64
Month: int64
DayofMonth: int64
DayOfWeek: int64
DepTime: double
CRSDepTime: int64
ArrTime: int64
CRSArrTime: int64
UniqueCarrier: string
FlightNum: int64
CRSElapsedTime: double
AirTime: int64
Origin: string
Dest: string
Distance: double
TaxiOut: double
Cancelled: int64
CarrierDelay: double
WeatherDelay: double
NASDelay: double
SecurityDelay: double
LateAircraftDelay: double
metadata
--------
OrderedDict([(b'pandas',
              b'{"index_columns": [{"kind": "range", "name": null, "start": '
              b'95550000, "stop": 95560000, "step": 1}], "column_indexes": ['
              b'{"name": null, "field_name": null, "pandas_type": "unicode",'
              b' "numpy_type": "object", "metadata": {"encoding": "UTF-8"}}]'
              b', "columns": [{"name": "Year", "field_name": "Year", "pandas'
              b'_type": "int64", "numpy_type": "int64", "metadata": nu

In [ ]:
df = dataset.read().to_pandas().set_index(['Year', 'Month'], inplace=True)

## cleanup

In [ ]:
import shutil
shutil.rmtree(TARGET_PATH)

### single parquet file

run this only when `dataset=False`

In [ ]:
assert KEY in run.outputs.keys(), f"mlrun.functions: key {KEY} not found in outputs"
assert os.path.isfile(TARGET_PATH+'/'+ FILE_NAME),  f"mlrun.functions: artifact source not found at {TARGET_PATH+'/'+ FILE_NAME}"

In [ ]:
copied   = pd.read_parquet(TARGET_PATH+'/'+ FILE_NAME, engine="pyarrow")